In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
import re
from operator import attrgetter, itemgetter
from functools import partial

import torch
import yake
import keybert
import pandas as pd
from tqdm import tqdm

from src.settings import DATA_PATH
from src.utils import compose
from src.morphy_utils import eq, Matcher

[INFO] Loading dictionaries from C:\Users\uiqko\pyenvs\mint\lib\site-packages\pymorphy2_dicts_ru\data
[INFO] format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [76]:
tqdm.pandas()

In [77]:
df = pd.read_csv(DATA_PATH / 'results' / 'tables.full.v4.1.csv')

In [78]:
df.set_index('FILE_NAME', inplace=True)

In [79]:
df.head(10)

,Планируемые результаты обучения,Наименование раздела дисциплины,Наименование раздела дисциплины (модуля),Наименование оценочного средства,Содержание
FILE_NAME,,,,,
(206813) 41 Инжиниринг программных систем.pdf.tables.json,производить проектирование и конструирование п...,"Конструирование программных систем,Анализ треб...","Конструирование программных систем,Курсовая ра...",NaN,NaN
(206815) 61 Когнитивная психология.pdf.tables.json,"различные категории когнитивной психологии,опт...",Психолингвистика и регуляция когнитивных проце...,Психолингвистика и регуляция когнитивных проце...,NaN,NaN
(206816) 60 Методы и средства для моделирования программных средств .pdf.tables.json,созданных при помощи различных методологий и п...,Моделирование ПС на основе информационного под...,Моделирование ПС на основе объектноориентирова...,NaN,NaN
(206822) 60 Основы проектирования информационных систем.pdf.tables.json,создавать модель по предъявляемому программном...,Проектирование ИС на основе функционального по...,Проектирование ИС на основе функционального по...,NaN,NaN
(206823) 57 Основы разработки компиляторов.pdf.tables.json,"и оптимальный способ представления данных,осно...",Генерация промежуточного представления програм...,Генерация промежуточного представления програм...,NaN,NaN
(206825) ГИА.pdf.tables.json,NaN,NaN,NaN,"Задание на ВКР,Предзащита ВКР,Отзыв руководите...",NaN
(206826) 42 Проектная документация.pdf.tables.json,"применяя действующие стандарты и нормативы,раз...",Интеграция и документирование информационных с...,Интеграция и документирование информационных с...,NaN,NaN
(206827) ПП Производственная преддипломная практика.pdf.tables.json,Создавать инструментальные средства программир...,NaN,NaN,NaN,NaN
(206828) П Производственная технологическая проектно-технологическая практика.pdf.tables.json,Создавать инструментальные средства программир...,NaN,NaN,NaN,NaN


In [81]:
df['all_keywords'] = df.fillna('').applymap(' '.__add__).sum(axis=1).apply(partial(re.sub, ',', ' '))

In [83]:
df.all_keywords.head(10)

FILE_NAME
(206813) 41 Инжиниринг программных систем.pdf.tables.json                                         производить проектирование и конструирование ...
(206815) 61 Когнитивная психология.pdf.tables.json                                                различные категории когнитивной психологии оп...
(206816) 60 Методы и средства для моделирования программных средств .pdf.tables.json              созданных при помощи различных методологий и ...
(206822) 60 Основы проектирования информационных систем.pdf.tables.json                           создавать модель по предъявляемому программно...
(206823) 57 Основы разработки компиляторов.pdf.tables.json                                        и оптимальный способ представления данных осн...
(206825) ГИА.pdf.tables.json                                                                         Задание на ВКР Предзащита ВКР Отзыв руково...
(206826) 42 Проектная документация.pdf.tables.json                                                применяя д

In [84]:
yake_extractor = yake.KeywordExtractor(lan='ru', n=5)

In [85]:
yake_df = df.all_keywords[:20].progress_apply(
    compose(yake_extractor.extract_keywords, partial(map, itemgetter(0)), ','.join))

100%|██████████| 20/20 [00:17<00:00,  1.14it/s]


In [86]:
yake_df[0]

'качество в установленное время Конструирование,Умения производить анализ требований Знания,Конструирование программных систем Анализ требований,Конструирование программных систем Курсовая работа,программных систем Курсовая работа Анализ,систем Конструирование программных систем Курсовая,поддержка программных систем Конструирование программных,искусственного интеллекта проводить тестирование программных,тестирование программных систем оценивать качество,систем Курсовая работа Анализ требований,проводить тестирование программных систем оценивать,анализ требований Знания алгоритмов используемых,производить анализ требований Знания алгоритмов,программных систем Конструирование программных систем,систем Умения производить анализ требований,алгоритмов искусственного интеллекта проводить тестирование,интеллекта проводить тестирование программных систем,параметров Навыки разработки концепции программных,информационных систем Умения производить анализ,программных систем оценивать качество прог

In [87]:
keybert_extractor = keybert.KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

[INFO] Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2
[INFO] Use pytorch device: cpu


In [88]:
def extract_keywords(d, kw):
    return d.progress_apply(
        compose(
            partial(keybert_extractor.extract_keywords, **kw),
            partial(map, attrgetter('word')),
            ','.join
        )
    )

In [89]:
extract_kwargs = dict(top_n=15)

In [98]:
kws = {
    'default': extract_kwargs,
    'range (1, 3)': (range_1_3 := dict(
        keyphrase_ngram_range=(1, 3),
        **extract_kwargs
    )),
    'mmr, range (1, 3)': (mmr := dict(
        use_mmr=True,
        **range_1_3
    )),
    'mmr, maxsum, range(1, 3)': dict(
        use_maxsum=True,
        **mmr
    ),
    'mmr, maxsum, range(2, 5)': dict(
        use_maxsum=True,
        use_mmr=True,
        keyphrase_ngram_range=(2, 5),
        **extract_kwargs,
    ),

}

In [99]:
for name, kwargs in kws.items():
    print(name)
    print('\n\n'.join(extract_keywords(df.all_keywords[:10], kwargs)))
    print()

default


100%|██████████| 10/10 [00:06<00:00,  1.66it/s]


алгоритмов,нейротехнологических,алгоритмы,программирования,биометрических,искусственного,интеллекта,программные,проектирование,программных,программной,методологий,данных,системах,инжиниринга

когнитивной,психологии,психолингвистика,когнитивного,когнитивных,психодиагностические,мозга,психологические,психологического,интеллектуально,интеллекта,восприятия,методологические,интеллектуального,методик

моделирование,моделей,модель,модели,объектноориентированного,методологий,программирования,подхода,математических,процессов,математическую,математические,программных,данных,математики

проектирование,проектирования,объектноориентированного,программирования,программный,программному,программного,моделирования,проектной,программ,разрабатывать,методологии,модель,скрипту,методологий

языков,программирования,алгоритмы,языковых,синтаксически,языкового,процессоров,синтаксических,языке,лексических,процессора,синтаксического,грамматик,дмппроцессоров,семантику

вкр,предзащита,руководителя,защита,отзыв,зада

100%|██████████| 10/10 [00:15<00:00,  1.58s/it]


системах искусственного интеллекта,алгоритмов искусственного интеллекта,искусственного интеллекта используемых,искусственного интеллекта биометрических,искусственного интеллекта проводить,искусственного интеллекта обработки,использованием алгоритмов искусственного,искусственного интеллекта,данных алгоритмов искусственного,интеллекта обработки биометрических,алгоритмы обработки биометрических,числе использованием алгоритмов,системы реализуя алгоритмы,систем использованием алгоритмов,знания алгоритмов используемых

подходов когнитивной психологии,категории когнитивной психологии,концепций когнитивной психологии,основы когнитивной психологии,когнитивной психологии оптимальному,когнитивной психологии особенности,развитию когнитивных процессов,развития когнитивных процессов,учетом когнитивной психологии,когнитивных процессов применять,когнитивной психологии,когнитивной психологии интеллекта,психолингвистика регуляция когнитивных,различные категории когнитивной,оптимальному развитию когнитив

...

системах искусственного интеллекта,проектирование конструирование программных,для нейротехнологических проектов,алгоритмы обработки биометрических,систем тестирование поддержка,числе использованием алгоритмов,анализ требований знания,систем умения производить,интеллекта проводить тестирование,инжиниринга информационных систем,программных систем анализ,используемых системах искусственного,биометрических параметров навыки,искусственного интеллекта биометрических,выбранном языке

подходов когнитивной психологии,диагностики уровня развития,проектировать интеллектуально инновационные,диагностических методик памяти,психологии оптимальному развитию,процессов сенсорные системы,основные психодиагностические методы,оптимальному развитию когнитивных,основных теорий концепций,развития личности методологические,системы психолингвистика регуляция,обработки интерпретации результатов,категории когнитивной,когнитивной деятельности проектировать,регуляция когнитивных процессов

сопоставлять модели прогр

100%|██████████| 10/10 [00:15<00:00,  1.56s/it]


системах искусственного интеллекта,проектирование конструирование программных,для нейротехнологических проектов,алгоритмы обработки биометрических,систем тестирование поддержка,числе использованием алгоритмов,анализ требований знания,систем умения производить,интеллекта проводить тестирование,инжиниринга информационных систем,программных систем анализ,используемых системах искусственного,биометрических параметров навыки,искусственного интеллекта биометрических,выбранном языке

подходов когнитивной психологии,диагностики уровня развития,проектировать интеллектуально инновационные,диагностических методик памяти,психологии оптимальному развитию,процессов сенсорные системы,основные психодиагностические методы,оптимальному развитию когнитивных,основных теорий концепций,развития личности методологические,системы психолингвистика регуляция,обработки интерпретации результатов,категории когнитивной,когнитивной деятельности проектировать,регуляция когнитивных процессов

сопоставлять модели прогр

100%|██████████| 10/10 [00:31<00:00,  3.11s/it]

числе использованием алгоритмов искусственного интеллекта,конструирование программных систем для нейротехнологических,биометрических параметров навыки разработки концепции,данных оценивать качество программной,системы реализуя алгоритмы обработки биометрических,интеллекта проводить тестирование программных систем,знания основных методологий инжиниринга информационных,систем анализ требований проектирование программных,системах искусственного,программирования программные системы реализуя алгоритмы,производить анализ требований знания алгоритмов,реализовывать на выбранном языке,интеллекта обработки биометрических данных знания,систем умения производить анализ,биометрических данных алгоритмов искусственного интеллекта

когнитивной психологии оптимальному развитию когнитивных,данных помощью сбора диагностических методик,исследования учетом основных теорий концепций,основные психодиагностические,результатов диагностики основные теории когнитивного,методы для исследования индивида развития,а

In [ ]:
text = 'системах искусственного интеллекта'.split()

In [93]:
from src.morphy_utils import decline_case, Matcher

In [ ]:
list(map(attrgetter('word'), decline_case(text, {('сущ', 'прил', 'сущ'): ('и', 'ignore', 'ignore')})[0]))

In [ ]:
from src.morphy_utils import morph_analyzer

In [153]:
morph_analyzer.parse('оптимальному')[0].inflect({'gent'})

Parse(word='оптимального', tag=OpencorporaTag('ADJF neut,sing,gent'), normal_form='оптимальный', score=1.0, methods_stack=((DictionaryAnalyzer(), 'оптимального', 60, 15),))

In [145]:
def decline_filter(text: str):
    text = text.split(',')
    res = []
    for keyword in text:
        declined = decline_case(keyword.split(), cases_by_mask=cases_by_mask)
        if declined:
            res.append(' '.join(map(attrgetter('word'), declined[0])))
    return res

[INFO] Loading dictionaries from C:\Users\uiqko\pyenvs\mint\lib\site-packages\pymorphy2_dicts_ru\data
[INFO] format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [ ]:
decline_filter(text)

In [161]:
test = extract_keywords(
    df.all_keywords[:10],
    dict(
        use_maxsum=True,
        use_mmr=True,
        keyphrase_ngram_range=(3, 3),
        **extract_kwargs
    )
).apply(decline_filter)

100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


In [162]:
test

FILE_NAME
(206813) 41 Инжиниринг программных систем.pdf.tables.json                                        [системы искусственного интеллекта, инжиниринг...
(206815) 61 Когнитивная психология.pdf.tables.json                                               [подходы когнитивной психологии, психология оп...
(206816) 60 Методы и средства для моделирования программных средств .pdf.tables.json             [программирование математических моделей, осно...
(206822) 60 Основы проектирования информационных систем.pdf.tables.json                                       [код функционального проектирования]
(206823) 57 Основы разработки компиляторов.pdf.tables.json                                               [проектирование лексических анализаторов]
(206825) ГИА.pdf.tables.json                                                                                                                    []
(206826) 42 Проектная документация.pdf.tables.json                                               [разработка

In [236]:
from pymorphy2.analyzer import Parse
from src.morphy_utils import _name_by_pos, Apply, _pos_by_name, _case_by_name

parse = compose(
    morph_analyzer.parse,
    itemgetter(0),
)

get_pos = compose(
    parse,
    attrgetter('tag'),
    attrgetter('POS'),
    _name_by_pos.get,
)

In [278]:
cases_by_mask = {
    ('сущ', 'прил', 'сущ'): ('и', 'р', 'р'),
    ('прил', 'сущ'): ('и', 'и'),
    ('сущ', 'сущ'): ('и', 'р'),
    ('сущ', 'сущ', 'прил', 'сущ'): ('и', 'р', 'р', 'р'),
    ('сущ',): ('и',)
}

In [279]:

mas = {}


def decline_or_ignore(word, case_name):
    case = _case_by_name[case_name]
    if case and (r := word.inflect({case[0]})):
        return r
    return word


for poses, cases in cases_by_mask.items():
    matcher = Matcher([
        (get_pos, eq(pos)) for pos in poses
    ])

    apply = Apply([
        (
            parse,
            partial(decline_or_ignore, case_name=case_name),
            attrgetter('word')
        ) for case_name in cases
    ])

    mas[matcher] = compose(apply, ' '.join)

def apply_if_match(text: str):
    text = text.split()
    for matcher, apply in mas.items():
        if matcher(text):
            a = apply(text)
            return a
    return None

In [296]:
best_kwargs = dict(
    top_n=50,
    keyphrase_ngram_range=(1, 3),
    use_mmr=True,
    use_maxsum=True,
    diversity=0.3,
)

In [292]:
small_df = df.all_keywords[:300].apply(partial(re.sub, '(знание|знания|умение|умения)', '', flags=re.IGNORECASE))

In [293]:
small_df

FILE_NAME
(206813) 41 Инжиниринг программных систем.pdf.tables.json                                производить проектирование и конструирование ...
(206815) 61 Когнитивная психология.pdf.tables.json                                       различные категории когнитивной психологии оп...
(206816) 60 Методы и средства для моделирования программных средств .pdf.tables.json     созданных при помощи различных методологий и ...
(206822) 60 Основы проектирования информационных систем.pdf.tables.json                  создавать модель по предъявляемому программно...
(206823) 57 Основы разработки компиляторов.pdf.tables.json                               и оптимальный способ представления данных осн...
                                                                                                              ...                        
(240275) Б1.62 Биотехнология микроорганизмов в производстве напитков.pdf.tables.json     охраны труда применять правила безопасной раб...
(240294) Б1.52_2 техноло

In [297]:
test = extract_keywords(
    small_df,
    best_kwargs
)

100%|██████████| 300/300 [11:17<00:00,  2.26s/it]


In [261]:
test

FILE_NAME
(206813) 41 Инжиниринг программных систем.pdf.tables.json                                        системах искусственного интеллекта,алгоритмы о...
(206815) 61 Когнитивная психология.pdf.tables.json                                               подходов когнитивной психологии,оптимальному р...
(206816) 60 Методы и средства для моделирования программных средств .pdf.tables.json             сопоставлять модели программных,математических...
(206822) 60 Основы проектирования информационных систем.pdf.tables.json                          объектного проектирования uml,методологий прое...
(206823) 57 Основы разработки компиляторов.pdf.tables.json                                       языкового процессора методы,конструкций языков...
(206825) ГИА.pdf.tables.json                                                                     отзыв руководителя вкр,вкр защита вкр,вкр пред...
(206826) 42 Проектная документация.pdf.tables.json                                               логического

In [299]:
test.apply(compose(
    partial(str.split, sep=','),
    partial(map, apply_if_match),
    partial(filter, bool),
    set,
    ','.join
)).to_csv(DATA_PATH / 'results' / 'keybert.small.v2.5.csv')